# Initial Model

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az

from cmdstanpy import cmdstan_path, CmdStanModel

In [7]:
PROJECT_DIR = "/home/george/parts/"
print(cmdstan_path())

/home/george/.cmdstanpy/cmdstan-2.21.0


## Data

In [13]:
df = pd.read_csv(os.path.join(PROJECT_DIR, "data", "tidy", "parts.csv"))

In [14]:
class_name = "analysis.ii"

friday_mask = (df.loc[:, "class"] == "analysis.ii") & (df.loc[:, "day"] == "fri")
friday_data = df[friday_mask].loc[:, "parts"].dropna().values
non_friday_data = df[~friday_mask].loc[:, "parts"].dropna().values
N = non_friday_data.size
M = friday_data.size

In [15]:
def run_stan_model(stan_file, data, **kwargs):
    """
    Convenience function to compile, sample and diagnose a Stan model.
    
    Notes
    -----
    For prior predictive sampling (or to otherwise simulate data), pass `fixed_param=True`.
        https://cmdstanpy.readthedocs.io/en/latest/sample.html#example-generate-data-fixed-param-true
    """
    model = CmdStanModel(stan_file=stan_file)
    model.compile()
    fit = model.sample(data=data, **kwargs)
    fit.diagnose()
    return model, fit

## Examine Prior Predictive Distribution

In [33]:
data = {
    "N": 100  # Number of samples
}

model, fit = run_stan_model(
    stan_file=os.path.join(PROJECT_DIR, "models", "_00-initial-prior_pred.stan"),
    data=data,
    fixed_param=True,
)

"""
cmdstanpy_data = az.from_cmdstanpy(
    posterior=fit,
    prior_predictive=["prior_pred_non_friday_parts", "prior_pred_friday_parts"],
    coords={'sample': np.arange(data["N"])},
    dims={
        "prior_pred_non_friday_parts": ["sample"],
        "prior_pred_friday_parts": ["sample"]
    }
)
""";

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /home/george/parts/models/_00-initial-prior_pred
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


Effective sample size satisfactory.

Split R-hat values satisfactory all parameters.

Processing complete, no problems detected.


We need to use a Betancourt plot, since plotting the density will confuse the _distribution_ of parts (which is what we care about) with the _predicted number_ of parts (which is what `az.plot_density` would show).

In [36]:
prior_pred_non_friday_parts = fit.get_drawset(["prior_pred_non_friday_parts"])
prior_pred_friday_parts = fit.get_drawset(["prior_pred_friday_parts"])

In [37]:
prior_pred_non_friday_parts

,prior_pred_non_friday_parts.1,prior_pred_non_friday_parts.2,prior_pred_non_friday_parts.3,prior_pred_non_friday_parts.4,prior_pred_non_friday_parts.5,prior_pred_non_friday_parts.6,prior_pred_non_friday_parts.7,prior_pred_non_friday_parts.8,prior_pred_non_friday_parts.9,prior_pred_non_friday_parts.10,...,prior_pred_non_friday_parts.91,prior_pred_non_friday_parts.92,prior_pred_non_friday_parts.93,prior_pred_non_friday_parts.94,prior_pred_non_friday_parts.95,prior_pred_non_friday_parts.96,prior_pred_non_friday_parts.97,prior_pred_non_friday_parts.98,prior_pred_non_friday_parts.99,prior_pred_non_friday_parts.100
0,19.0,17.0,16.0,24.0,16.0,20.0,19.0,15.0,23.0,13.0,...,16.0,12.0,25.0,10.0,15.0,14.0,18.0,9.0,22.0,19.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,18.0,17.0,23.0,21.0,18.0,19.0,14.0,19.0,8.0,15.0,...,25.0,15.0,20.0,18.0,18.0,19.0,24.0,15.0,19.0,25.0
3,8.0,6.0,9.0,8.0,12.0,3.0,8.0,12.0,8.0,7.0,...,4.0,10.0,10.0,8.0,6.0,3.0,7.0,9.0,5.0,6.0
4,7.0,2.0,3.0,2.0,7.0,3.0,1.0,2.0,1.0,2.0,...,4.0,3.0,3.0,4.0,3.0,6.0,1.0,2.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,4.0,12.0,5.0,3.0,5.0,4.0,6.0,4.0,8.0,4.0,...,3.0,5.0,5.0,5.0,4.0,4.0,2.0,4.0,7.0,6.0
996,12.0,8.0,12.0,10.0,16.0,5.0,16.0,16.0,12.0,10.0,...,17.0,18.0,16.0,14.0,11.0,22.0,14.0,19.0,12.0,14.0
997,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,2.0,...,0.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
998,2.0,7.0,6.0,5.0,8.0,6.0,6.0,5.0,8.0,6.0,...,10.0,6.0,8.0,5.0,7.0,11.0,7.0,7.0,7.0,6.0


## Fit Model